In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import datetime
#from tensorboard.plugins.hparams import api as hp
from sklearn.model_selection import train_test_split
import cv2
from os import listdir
from tensorflow import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from sklearn.preprocessing import LabelBinarizer
import random
from sklearn.utils import shuffle
import cv2
import imutils

Using TensorFlow backend.


In [8]:
def resize_image(image, width, height):

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    return image

In [9]:
files = [f for f in listdir('ready')]

print(len(files))

X_all = []
y_all = []

for file in shuffle(files):
    image = cv2.imread('ready/'+file)

    # convert image to only 0 or 255
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # resize image
    image = resize_image(image, 12, 12)

    #first char is the label
    y_all.append(file[0])

    X_all.append(image)

X_all = np.array(X_all)

# reshape array
X_all = X_all.reshape((X_all.shape[0],12,12,1))

# convert it to either 1 or 0
X_all = np.array(X_all,dtype='float') /255.0

# cast y to array
y_all = np.array(y_all)

X_train, X_test, y_train, y_test = train_test_split(X_all,y_all, test_size=0.2, random_state = 42)

#encode to one hot encoding
lb = LabelBinarizer().fit(y_train)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

807


In [10]:
# Build the neural network!
model = Sequential()

# First convolutional layer with max pooling
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(X_all.shape[1], X_all.shape[2],1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(200, activation="relu"))

# Output layer with 32 nodes (one for each possible letter/number we predict)
model.add(Dense(10, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

model.fit(X_train, y_train, batch_size=24, validation_data=(X_test, y_test), epochs=30, verbose=1)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 12, 12, 20)        520       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 20)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 720)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 200)               144200    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                2010      
Total params: 146,730
Trainable params: 146,730
Non-trainable params: 0
_________________________________________________________________
None
Train on 645 samples, validate on 162 samples
Epoch 1/30
645/645 [==============================] - 1s 976us/

In [11]:
model.save('model.hdf5')